# Bipartite Overview

https://networkx.org/documentation/stable/reference/algorithms/bipartite.html?highlight=bipartite#module-networkx.algorithms.bipartite

In [1]:
from networkx.algorithms import bipartite
import networkx as nx
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')



In [2]:
with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [3]:
B = nx.Graph()

In [4]:
# Add nodes with the node attribute "bipartite"

B.add_nodes_from([1, 2, 3, 4], bipartite=0)

B.add_nodes_from(["a", "b", "c"], bipartite=1)

# Add edges only between nodes of opposite node sets

B.add_edges_from([(1, "a"), (1, "b"), (2, "b"), (2, "c"), (3, "c"), (4, "a")])

In [5]:
nx.is_connected(B)


True

In [6]:


bottom_nodes, top_nodes = bipartite.sets(B)

In [7]:
top_nodes = {n for n, d in B.nodes(data=True) if d["bipartite"] == 0}

bottom_nodes = set(B) - top_nodes

In [8]:
print(round(bipartite.density(B, bottom_nodes), 2))

0.5


In [9]:
G = bipartite.projected_graph(B, top_nodes)

In [10]:
RB = bipartite.random_graph(5, 7, 0.2)

RB_top = {n for n, d in RB.nodes(data=True) if d["bipartite"] == 0}

RB_bottom = set(RB) - RB_top

list(RB_top)




[0, 1, 2, 3, 4]

In [11]:
list(RB_bottom)

[5, 6, 7, 8, 9, 10, 11]

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.edgelist.read_edgelist.html#networkx.algorithms.bipartite.edgelist.read_edgelist

# Getting Actual Augur Network Data

In [15]:
contrib_net = pd.DataFrame()

net_query = salc.sql.text(f"""
            SELECT 
                a.cntrb_id,
                a.repo_id,
                b.repo_name,
                b.repo_git,
                a.login, 
                MAX ( RANK ) AS weight 
            FROM
                explorer_contributor_actions a,
                repo b 
            WHERE
                a.repo_id = b.repo_id 
            GROUP BY
                a.cntrb_id,
                a.repo_id,
                b.repo_name,
                b.repo_git,
                a."login"
            ORDER BY
                repo_id;  

                """)

network = pd.read_sql(net_query, con=engine)

display(network)

network.dtypes

,cntrb_id,repo_id,repo_name,repo_git,login,weight
0,25537,1,rails,https://github.com/rails/rails.git,jcoyne,49
1,25566,1,rails,https://github.com/rails/rails.git,jeremyf,8
2,25577,1,rails,https://github.com/rails/rails.git,cbeer,1
3,25635,1,rails,https://github.com/rails/rails.git,twalpole,10
4,25952,1,rails,https://github.com/rails/rails.git,nicolasfranck,1
...,...,...,...,...,...,...
22322,25640,26093,xslt,https://github.com/ropensci/xslt,cboettig,1
22323,25678,26093,xslt,https://github.com/ropensci/xslt,jimhester,3
22324,25717,26093,xslt,https://github.com/ropensci/xslt,zkamvar,1
22325,42393,26093,xslt,https://github.com/ropensci/xslt,oggioniale,1


cntrb_id      int64
repo_id       int64
repo_name    object
repo_git     object
login        object
weight        int64
dtype: object

In [16]:
augNet = nx.Graph()

In [17]:
# Add distinct repo ids
# Add distinct contrib ids
# Add edges between them 

In [23]:
nodecontrb = network['cntrb_id'].unique()

In [25]:
print(nodecontrb)

[25537 25566 25577 ... 51451 51459 51433]


In [31]:
noderepo = network['repo_git'].unique()

In [35]:
print(noderepo)


['https://github.com/rails/rails.git'
 'https://github.com/senaite/senaite.core.git'
 'https://github.com/senaite/senaite.health.git'
 'https://github.com/senaite/senaite.lims.git'
 'https://github.com/senaite/senaite.com.git'
 'https://github.com/senaite/senaite.github.io.git'
 'https://github.com/senaite/senaite.api.git'
 'https://github.com/senaite/senaite.jsonapi.git'
 'https://github.com/senaite/senaite.sync.git'
 'https://github.com/senaite/senaite.exporter.git'
 'https://github.com/senaite/senaite.impress.git'
 'https://github.com/senaite/senaite.core.supermodel.git'
 'https://github.com/senaite/senaite.databox.git'
 'https://github.com/senaite/senaite.reflex.git'
 'https://github.com/senaite/senaite.storage.git'
 'https://github.com/senaite/senaite.core.listing.git'
 'https://github.com/senaite/senaite.instruments.git'
 'https://github.com/senaite/senaite.docker.git'
 'https://github.com/mollyproject/mollyproject.git'
 'https://github.com/mollyproject/django-compress.git'
 'htt

In [36]:
augNet.add_nodes_from(nodecontrb, bipartite=0)

In [37]:
augNet.add_nodes_from[noderepo, bipartite=1)

SyntaxError: invalid syntax (464204366.py, line 1)

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [40]:
https://www.geeksforgeeks.org/numpy-ndarray/

SyntaxError: invalid syntax (3032881049.py, line 1)